In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests as rq

In [2]:
title=[]
genre=[]
censor_rating=[]
ratings=[]
year=[]
actors=[]
characters=[]
overview=[]

In [3]:
#functions to handle Nan values if any
def fetcher(soup,obj,Class=None,gett=None):
    if Class==None:
        if soup.find(obj):
            return soup.find(obj).text
    elif gett == None:
        if soup.find(obj,class_=Class):
            return (soup.find(obj,class_=Class).text).replace('\n','')
    else:
        if soup.find(obj,class_=Class):
            return soup.find(obj,class_=Class).get(gett)
    return np.nan


def fetcher1(soup,obj,key,value):
    if soup.find(obj,{key:value}):
        return soup.find(obj,{key:value}).text
    else:
        return np.nan

In [4]:
soup=BeautifulSoup()

In [5]:
for a in range(1990,2023): #for all years between 1990 to 2022
    for j in range(1,452,50): #fetching top 500 popular movies of each year
        req=rq.get(f'https://www.imdb.com/search/title/?year={a}-01-01,{a}-12-31&start={j}&ref_=adv_nxt').text
        soup=BeautifulSoup(req,'lxml')
        x=soup.find_all('div',class_='lister-item')#main page
        
        for i in x:
            header=i.find('h3',class_='lister-item-header')
            title.append(fetcher(header,'a'))
            link=header.find('a').get('href')
            year.append(fetcher(i,'span','lister-item-year'))
            censor_rating.append(fetcher(i,'span','certificate'))
            ratings.append(fetcher(i,'div','ratings-imdb-rating','data-value'))
            genre.append(fetcher(i,'span','genre'))
            
            res2=rq.get('https://www.imdb.com/'+link).text#for getting into child page (specific for each movie)
            s2=BeautifulSoup(res2,'lxml')
            info=s2.find_all('div',{'data-testid':'title-cast-item'})
            actor=''
            character=''
            for i in range(min(5,len(info))):#run for top 5 entries or for maximum no of entries if they are less than 5
                s=info[i]
                actor+=str(fetcher1(s,'a','data-testid',"title-cast-item__actor"))+', '
                character+=str(fetcher(s,'span'))+', '
            actors.append(actor[:-2])
            characters.append(character[:-2])
            overview.append(fetcher1(s2,'span','data-testid',"plot-xs_to_m"))

In [6]:
df=pd.DataFrame({'Title':title,'Year':year,'Ratings': ratings,'Genre':genre,'Overview':overview,'Actors':actors,'Characters':characters,'Censorship':censor_rating})

In [7]:
df.to_csv('IMDB_data_final.csv',index=False)